In [ ]:
### ZERO-SHOT CLASSIFICATION, TOPIC LABELLING ###

# TODOs: Discovering Interpretable Topics by Leveraging Common Sense Knowledge (Facebook)
# TODOs: https://towardsdatascience.com/zero-shot-text-classification-with-hugging-face-7f533ba83cd6
# DONE: Extract args, ids for labelling
# DONE: Get Labels
# TODOs: NLI vs Semantic
# TODOs: Zero-shot Classification ConceptNet (semantic extension)
# TODOs: Expand: Concept Net synonyms

In [1]:
### ASSERT WORKING DIRECTORY FOR IMPORTS ###
import os
print(os.getcwd())

/Users/joshua.sheppard/PycharmProjects/countaBot/src/detection


In [2]:
import pandas as pd
import json
import random
from tqdm.notebook import tqdm

In [3]:
### LOAD ###
#args = [json.loads(ln) for ln in open("../data/cmv_processed.jsonl")]
ids = [json.loads(ln)["id"] for ln in open("../data/cmv_processed.jsonl")]
args = [json.loads(ln)["argument"]["argument"] for ln in open("../data/cmv_processed.jsonl")]

len(args), len(ids)

(10303, 10303)

In [4]:
for j, k in zip(args, ids):
    if j == "":
        print("blanks", j, k)

blanks  t3_3cm6jy
blanks  t3_1egv4k
blanks  t3_1egv4k
blanks  t3_5wjdve


In [5]:
### SAMPLE ###
_ = random.randint(0, len(args))
args[_]

'Its simple. I think society is allows people to get away with colossal gluttony way too often. CEOs make millions a year music artists even more so. Its just a massive cabal of rich people getting richer. Money More Money for them this shouldnt be true for people who make million or more.'

In [6]:
# Combined Label Set - CauseNet
# Experiment Title vs Argument
# Eval 1: Arguments

corpus = set()
idx = set()

for j, k in zip(args, ids):
    corpus.add((j, k))

corpus = list(corpus)
type(corpus), len(corpus)

(list, 5990)

In [7]:
### COMMON SENSE TOPIC LABELS ###
import re
def clean(clean):
    clean = re.sub(r"\n", "", clean)
    clean = re.sub(r'(?<=[a-z])\'(?=[a-z])', '', clean)
    clean = re.sub('([^a-zA-Z\s.!?])', "", clean)
    clean = re.sub('\s+', ' ', clean)

    clean = re.sub(r"www\S+", "", clean)
    return clean.strip().lower()

#TODOs: Prune and Add CauseNet Topics
controversial_topics = [clean(ln) for ln in open("../data/concepts/wiki_controversial_topics.txt")]
debate_topics = [clean(ln) for ln in open("../data/concepts/IBM_debate_topics_I.txt")]
debate_topics_ = [clean(ln) for ln in open("../data/concepts/IBM_debate_topics_II.txt")]
arg_kb_20 = [clean(ln) for ln in open("../data/concepts/argkb_20_topics.txt")]

concepts = [clean(ln) for ln in open("../data/concepts/cause_concepts.txt")]

topics = []
topics.extend(controversial_topics)
topics.extend(debate_topics)
topics.extend(debate_topics_)
topics.extend(arg_kb_20)

len(topics)

6260

In [8]:
topics

['invasion of iraq',
 'abortion',
 'affirmative action',
 'african national congress',
 'alberta separatism',
 'american civil liberties union',
 'american hunters and shooters association',
 'history of the jews in the united states',
 'american jews',
 'anarchism',
 'anarchocapitalism',
 'antiamericanism',
 'anticlericalism',
 'antiirish racism',
 'antisemitism',
 'asian american',
 'atheism',
 'austrian school',
 'bashar assad',
 'syrian civil war',
 'bidoon social class',
 'black lives matter',
 'black supremacy',
 'boricua popular army',
 'brexit',
 'british national party',
 'capitalism',
 'capital punishment',
 'catalonia',
 'cbc news',
 'cherokee',
 'china',
 'east turkestan independence movement',
 'tibetan independence movement',
 'human rights in china',
 'chinese intelligence activity abroad',
 'christian right',
 'plame affair',
 'plame affair timeline',
 'communism',
 'communist party of china',
 'communist government',
 'communist state',
 'cnn',
 'conservatism in the un

In [9]:
# Semantic-Search, Cosine Similarity
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('all-MiniLM-L6-v2')

def semantic_search(corpus, query, threshold=0.15):
    # Construct Corpus set
    corpus_, id_ = zip(*corpus)
    corpus_ = list(corpus_)
    id_ = list(id_)

    # Embed the Corpus
    corpus_embeddings = embedder.encode(corpus_, convert_to_tensor=True)

    # Construct Query-Label set
    queries = set(query)

    mapped_dict = {}
    for i in range(0, len(id_)):
        mapped_dict[id_[i]] = {"argument": corpus_[i], "label": []}

    # Return top k=1 argument for each Label via Cosine Similarity
    top_k = min(1, len(corpus_))

    with tqdm(total=len(queries)) as pbar:
        for query in queries:
            query_embedding = embedder.encode(query, convert_to_tensor=True)

            cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
            top_results = torch.topk(cos_scores, k=top_k)

            for score, idx in zip(top_results[0], top_results[1]):
                # 'Empirical' threshold
                if score >= threshold:
                    # Append Label
                    #mapped_dict[id_]["argument"] = corpus_[idx]
                    # Note: Can use the same idx index
                    mapped_dict[id_[idx]]["label"] = query.lower()

                #else: mapped_dict[id_]["label"] = "None"

            pbar.update()

    return mapped_dict

In [10]:
### EXTRACT LABELLED INSTANCES ###
def extract_labelled(dict_):
    res = []
    for i in dict_.items():
        j, k = i

        if k["label"] != []:
            res.append({"id": i[0], "argument": k["argument"], "label": k["label"]})
    return res

In [11]:
# TODOs: Reverse the Query
#sample = corpus[0:100]

### LABELLING: CONTROVERSIAL TOPICS ###
arg_topics = semantic_search(corpus=corpus, query=topics, threshold=0)

  0%|          | 0/5464 [00:00<?, ?it/s]

In [12]:
print(len(arg_topics), len(corpus))
print(len(arg_topics), len(extract_labelled(arg_topics)))
arg_topics

5990 5990
5990 2224


{'t3_5c9vp9': {'argument': 'The first part should be easier to than the second part since all youve got to do is show an example where we have regressed on a federal level. And by regressed I mean a situation where rights were legally granted either by legislation or clarified as a result of a Supreme Court decision and then at some later date those rights were taken away at the federal level.A few caveats Dont tell me about the state representative that tried to make interracial marriage illegal in your state that isnt on the federal level. Dont tell me about state laws that limit access to abortion. Thats not only at the state level but isnt removing a right just making it more difficult to exercise. Im going to carve out prohibition.',
  'label': []},
 't3_56agcz': {'argument': 'I think especially Black Americans need to stop using racism as excuse for their personal shortcoming. I really dont know what they have to complain about. We have had a black president for years Quite succe

In [13]:
### LABELLING: CONCEPTS ###
arg_concepts = semantic_search(corpus=corpus, query=concepts)

  0%|          | 0/51359 [00:00<?, ?it/s]

In [14]:
len(arg_concepts), len(extract_labelled(arg_concepts))

(5990, 4093)

In [132]:
### STORE TOPIC LABELS CONCEPTS AND LABELS ###
fout = open("../data/argument_topic_concept.jsonl", "w")

for j, k in arg_topics.items():
    fout.write(json.dumps({
        "id": j,
        "argument": k["argument"],
        "topic_label": k["label"],
        #"concept_label": k["label"]
    }))
    fout.write("\n")

In [16]:
### STORE CONCEPT LABELS CONCEPTS AND LABELS ###
fout = open("../data/argument_concepts.jsonl", "w")

for j, k in arg_concepts.items():
    fout.write(json.dumps({
        "id": j,
        "argument": k["argument"],
        "concept_label": k["label"],
    }))
    fout.write("\n")

In [127]:
for j, k in arg_topics.items():
    if k["argument"] == "":
        print("blanks", j)

blanks t3_1egv4k
blanks t3_3cm6jy
blanks t3_5wjdve


In [103]:
arg_concepts

{'t3_6dj73v': {'argument': 'Pizza is wonderful. Top food stuff of mine easily but I do not understand the trend for thicker and thicker bases. I think of the base of a pizza as a vector for getting the delicious flavors of the sauce and the cheese and the toppings into my mouth. I dont want that wonderful tomato sauce goodness overpowered by soggy tasteless bread. The thinner base also means the pizza is crispier and you dont need to support the end of the slice with your other hand.',
  'label': []},
 't3_6i4hip': {'argument': 'Just a heads up Im gonna curse for a minute.I have been a hardcore gamer for the past ten years but my discourse with internet communities surrounding the games I play really began when I started playing League of Legends around years ago. Over those four years I have become more and more skeptical and ultimately completely disdainful of not only the people who play the online games I do but in general the gaming publics absolutely childlike reactions towards g

In [392]:
# ### QUERY EXPANSION ###
#
# # TODOs: Query Expansions [TypeOf, SimilarTerms, CanBe]
# # https://github.com/fitosegrera/python-conceptnet/blob/master/ConceptNet.py
# import json
# import urllib
#
# URL = "http://api.conceptnet.io/"
#
# # TODOs: Review. Similarity.
# class ConceptNet:
#
#     def __init__(self, api, l):
#         self.api = api
#         self.l = l
#
#     def search(self, lang, term):
#         url_to_search = self.api + "c/" + lang + "/" + term
#         data = urllib.request.urlopen(url_to_search)
#         json_data = json.load(data)
#         for i in json_data["edges"]:
#             print("----------------")
#             print(i["end"])
#             print("relation:", i["rel"])
#             print(i["surfaceEnd"])
#             print(i["surfaceStart"])
#             print("weight:", i["weight"])
#
#         return json_data
#
#     def get_relation(self, rel, concept):
#         url_to_search = self.api + f"search?node=/c/en/{concept}&rel=/r/{rel}"
#         data = urllib.request.urlopen(url_to_search)
#         obj_ = json.load(data)
#
#         labels = set()
#         for _ in obj_["edges"]:
#             labels.add((_["end"]["label"], _["weight"]))
#
#         return labels
#
#     def get_similar(self, concept):
#         res = []
#         rels = ["Synonym", "SimilarTo"]
#         for _ in rels:
#             res.extend(self.get_relation(_, concept))
#
#         return sorted(res, key=lambda x: x[1], reverse=True)[:self.l]

In [393]:
# concept_net = ConceptNet(api=URL, l=3)

In [90]:
### CONCEPT NET ###
# from src.utils_.concept_net_expansion import ConceptNet
# URL = "http://api.conceptnet.io/"
#
# concept_net = ConceptNet(URL, l=3)
# expansion_term = "carriage"
#
# test_concept = concept_net.get_similar(expansion_term)
# test_concept

[('carriage', 2.0), ('coach', 1.0), ('carriage', 1.0)]

In [404]:
### WORDNET ###
# from src.utils_.word_net_expansion import expand_query
#
# test_wn = expand_query(expansion_term)
# test_wn

'fabulous fab mythic'

In [134]:
### EXTRACT LABELLED INSTANCES ###
# def extract_labelled(dict_):
#     res = []
#     for i in dict_.items():
#         j, k = i
#
#         if k["label"] != "None":
#             res.append({"id": i[0], "argument": k["argument"], "label": k["label"]})
#     return res

# args_topic_labelled = extract_labelled(arg_topics)
# print(len(args_topic_labelled))
#
# args_topic_labelled

In [135]:
# args_topic_labelled = extract_labelled(arg_topics)
# print(len(args_topic_labelled))
#
# args_topic_labelled

4782


[{'id': 't3_6d8okw',
  'argument': 'Casebycase schools that are largely successful are correlatively successful with their local schools compared to national peers. The mindset of the community matters. Many attribute the ailing innercity schools to cultural issues and biases having worked with innercity populations for five years and having worked with hundreds of students perfectly capable of rational thought and argument that nevertheless perform poorly I agree. In general American culture devalues intelligence some areas more than others. Literacy movements are wonderful but until people stop seeing learning as lame or avoiding intellectual discourse this wont change.',
  'label': 'Year-round school in the United States'},
 {'id': 't3_25fjdx',
  'argument': 'Ah like it says in Dust In The Wind. Every line in that song is correct except for nothing lasts forever but the earth and sky. Probably wont last forever either.I believe that it doesnt matter what you do in life. Well not in 